In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.linear_model as linear_model
import seaborn as sns
#import xgboost as xgb
from sklearn.model_selection import KFold
from IPython.display import HTML, display
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [ ]:
#df = pd.read_csv('./LoanStats3a.csv')
df = pd.read_csv('./Final_Dataset.csv')
df_origin=df

In [ ]:
df.info()

In [ ]:
#Inspecting the Null values
df.isnull().sum(axis=0)

In [ ]:
total_num = df.isnull().sum().sort_values(ascending=False)

perc = df.isnull().sum()/df.isnull().count() *100
perc1 = (round(perc,2).sort_values(ascending=False))

# Creating a data frame:
df_miss = pd.concat([total_num, perc1], axis =1 , keys =["Total Missing Values", "Percentage %"]).sort_values(by ="Percentage %", ascending = False)

In [ ]:
top_mis = df_miss[df_miss["Percentage %"]>0]
top_mis.reset_index(inplace=True)
top_mis

In [ ]:
df.shape


In [ ]:
y=df['Loss on Dispostion']

In [ ]:
add=df_origin[['App1RiskScore','App2RiskScore','Customers_Applicant_Gender','Vehicle_Trim','ProductSeries','DealerRegion','Customers_Applicant_EmpStatus','Customers_Applicant_EmpProvince',
'CashSalePrice','Customers_CoApplicant_AnnualTotal']]

In [ ]:
df=df.dropna(axis=1)


In [ ]:
df=df.join(add)
df=df.join(y)

In [ ]:
df['CashSalePrice'].fillna(df['CashSalePrice'].mean(), inplace=True)

In [ ]:
df.shape


In [ ]:
# lists number of unique values by each column
df.apply(lambda x: x.nunique())

In [ ]:
# removing the columns which has single(unique) values like - NA, 0, 'f', 'n' etc.
unique = df.nunique()
unique = unique[unique.values == 1]
unique

In [ ]:
# removing the columns which has single(unique) values like - NA, 0, 'f', 'n' etc.
df.drop(labels = list(unique.index), axis =1, inplace=True)

In [ ]:
df.info()

In [ ]:
df['Customers_CoApplicant_AnnualTotal']=df.Customers_CoApplicant_AnnualTotal.fillna(0)

In [ ]:
df['Customers_CoApplicant_AnnualTotal']

In [ ]:
df['Customers_Applicant_AnnualTotal']

In [ ]:
df['Customers_Applicant_AnnualTotal']=df['Customers_Applicant_AnnualTotal'] + df['Customers_CoApplicant_AnnualTotal']

In [ ]:
# 둘이 공동 대출이므로 하나의 건으로 본다.
df['Customers_Applicant_AnnualTotal']

In [ ]:
# 공동대출 그룹과 개인 대출 그룹의 비교를 위해 바이너리 변수 하나를 만든다.
df['Customers_CoApplicant'] =np.where(df['Customers_CoApplicant_AnnualTotal']>=1, 1, 0)

In [ ]:
df=df.drop('Customers_CoApplicant_AnnualTotal', axis=1)

In [ ]:
df.info()

In [ ]:
# object 타입변수들 unique 개수 봐서 필요없는것 삭제
df.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

In [ ]:
df.drop(['Customers_Applicant_PostalCode','CreatedOn','ApplicationNumber'], inplace=True, axis=1, errors='ignore')

In [ ]:
#연속성 변수 상관관계로 다중공선선 삭제

In [ ]:
plt.figure(figsize=(10, 8)) 
plt.figure(1)
corr = df.corr()
sns.heatmap(corr)


In [ ]:
corr.loc[:,:] = np.tril(corr, k=-1)   # below main lower triangle of an array
corr = corr.stack()
corr[(corr > 0.65) | (corr < -0.65)]


In [ ]:
df.drop(['Customers_Applicant_ApplicationID','DecisionID', 'Decisions_Vehicles_Losses_ApplicationID','Decisions_Booked_isLastDecision_Distinct_DealerID','PortalApplicationID','App1RiskScore','App2RiskScore',
       'Vehicles_Booked_Losses_DealerID','ProductYear','Vehicle_Year', 'ProductMileage','AmountToFinance', 'CoApplicantFlag','MonthlyIncome','LenderBookValue','LastAmountToFinance','PaymentFrequency','LTV_Value','InitialNumberOfApplicants'], inplace=True, axis=1, errors='ignore')

In [ ]:
df.columns

In [ ]:
plt.figure(figsize=(10, 8)) 
plt.figure(1)
corr = df.corr()

sns.heatmap(corr, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10})

In [ ]:
df.drop(['Customers_Applicant_Province','ProductMake', 'ProductModel','LastRiskTierName','RiskTierCode','LastRiskTierName','CustomerProvince','Vehicle_Trim','DealerRegion','Customers_Applicant_EmpProvince','RiskTierName'], inplace=True, axis=1, errors='ignore')

In [ ]:
df.info()

In [ ]:
df

In [ ]:
df.columns

In [ ]:
loss=df['Loss on Dispostion'].dropna()
loss

In [ ]:
loss.describe()

In [ ]:
from scipy import stats
from scipy.stats import norm, skew #for some statistics

sns.distplot(loss , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(loss)
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('distribution')

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(loss, plot=plt)
plt.show()

In [ ]:
df[df['Loss on Dispostion']>0]

In [ ]:
# # default net loss : 1, Besides that: 0 
# df['Loss on Dispostion'] =np.where(df['Loss on Dispostion']>0, 1, 0)
# df

In [ ]:
# default :1, normal:0
df['Loss on Dispostion'] = np.where(pd.notnull(df['Loss on Dispostion']), 1, 0)
df.head()

In [ ]:
fig = plt.figure(figsize=(18,18))

ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(212)

cmap = plt.cm.coolwarm_r

loans_by_region = df.groupby(['RiskTier', 'Loss on Dispostion']).size()
loans_by_region.unstack().plot(kind='bar', stacked=True, colormap=cmap, ax=ax1, grid=False)
ax1.set_title('Type of Loans by RiskTier', fontsize=14)


loans_by_grade = df.groupby(['Customers_Applicant_ResidentType', 'Loss on Dispostion']).size()
loans_by_grade.unstack().plot(kind='bar', stacked=True, colormap=cmap, ax=ax2, grid=False)
ax2.set_title('Type of Loans by ResidentType', fontsize=14)

by_interest = df.groupby(['Term', 'Loss on Dispostion']).InterestRate.mean()
by_interest.unstack().plot(ax=ax3, colormap=cmap)
ax3.set_title('Average Interest rate by Loan Condition', fontsize=14)
ax3.set_ylabel('Interest Rate (%)', fontsize=12)

In [ ]:
df.columns

In [ ]:
df.InterestRate.mean()

In [ ]:
# Average interest is 25% Anything above this will be considered of high risk let's see if this is true.
df['interest_payments'] = np.nan
lst = [df]

for col in lst:
    col.loc[col['InterestRate'] <= 25, 'interest_payments'] = 'Low'
    col.loc[col['InterestRate'] > 25, 'interest_payments'] = 'High'

In [ ]:
from scipy.stats import norm

plt.figure(figsize=(20,10))

palette = ['#009393', '#930000']
plt.subplot(221)
ax = sns.countplot(x='interest_payments', data=df, 
                  palette=palette, hue='Loss on Dispostion')

ax.set_title('The impact of interest rate \n on the condition of the loan', fontsize=14)
ax.set_xlabel('Level of Interest Payments', fontsize=12)
ax.set_ylabel('Count')

plt.subplot(222)
ax1 = sns.countplot(x='interest_payments', data=df, 
                   palette=palette, hue='Customers_Applicant_Gender')

ax1.set_title('The impact of interest rate \n on status of sex', fontsize=14)
ax1.set_xlabel('Level of Interest Payments', fontsize=12)
ax1.set_ylabel('Count')


plt.subplot(212)
low = df['TotalCalculatedDebt'].loc[df['interest_payments'] == 'Low'].values
high = df['TotalCalculatedDebt'].loc[df['interest_payments'] == 'High'].values


ax2= sns.distplot(low, color='#009393', label='Low Interest Payments', fit=norm, fit_kws={"color":"#483d8b"}) # Dark Blue Norm Color
ax3 = sns.distplot(high, color='#930000', label='High Interest Payments', fit=norm, fit_kws={"color":"#c71585"}) #  Red Norm Color
plt.axis([0, 36000, 0, 0.00016])
plt.legend()


plt.show()

In [ ]:
y=df['Loss on Dispostion']
x=df.iloc[:,:-1]

In [ ]:
quantitative = [f for f in df.columns if df.dtypes[f] != 'object']
qualitative = [f for f in df.columns if df.dtypes[f] == 'object']

In [ ]:
# missing categories imoutation
for c in qualitative:
    df[c] = df[c].astype('category')
    if df[c].isnull().any():
        df[c] = df[c].cat.add_categories(['MISSING'])
        df[c] = df[c].fillna('MISSING')

In [ ]:
# Encording

from scipy.stats import trim_mean
# m = stats.trim_mean(train[''], 0.1) # Trim 10% at both ends of the mean 극단치 제외하기위해 mean 대신 trim_mean 사용
    
def encode(frame, feature):
    ordering = pd.DataFrame()
    ordering['val'] = frame[feature].unique() #각 특성의 카테고리값을 val에 넣었다
    ordering.index = ordering.val
    ordering['spmean'] = frame[[feature, 'Loss on Dispostion']].groupby(feature)['Loss on Dispostion'].apply(trim_mean, 0.1) #각 특성의 카테고리값의 credit_score 평균을 spmean에 넣는다
    ordering = ordering.sort_values('spmean') # credit_score 평균값이 작은 카테고리부터 sort된다.
    ordering['ordering'] = range(1, ordering.shape[0]+1) # 1부터 카데고리수만큼 순서를 정한후
    ordering = ordering['ordering'].to_dict() #딕셔너리 키밸류 셋으로 만든후
    
    for cat, o in ordering.items(): # 키,밸류 뽑아서
        frame.loc[frame[feature] == cat, feature+'_E'] = o # 카테고리 이름과 변수 이름이 만나는 행렬의 밸류값 즉 credit_score평균값을 꺼낸다.
    
qual_encoded_C = []
for q in qualitative:  
    encode(df, q)
    qual_encoded_C.append(q+'_E')
print(qual_encoded_C)

In [ ]:
df.info()

In [ ]:
# 인코딩한 데이터로 대체해서 df 새로 만듬
df_C=df[quantitative+qual_encoded_C]

In [ ]:
df_C.info()

In [ ]:
def anova(frame):
    anv = pd.DataFrame()
    anv['feature'] = qualitative #질적 변수들을 사용
    pvals = []
    for c in qualitative:
        samples = []
        for cls in frame[c].unique(): # 각 독립변수의 카테고리들간의 SalePrice 평균값의 차이가 유의미 한지 ANOVA로 분석
            s = frame[frame[c] == cls]['Loss on Dispostion'].values
            samples.append(s)
        pval = stats.f_oneway(*samples)[1]
        pvals.append(pval)
    anv['pval'] = pvals 
    return anv.sort_values('pval') 

a = anova(df)
a['disparity'] = np.log(1./a['pval'].values) # p-value를 역으로 해서 차이가 큰거부터 줄어들도록 표시.
sns.barplot(data=a, x='feature', y='disparity')
x=plt.xticks(rotation=90)

#p-value 작은거 부터 왼쪽에 오도록 한다. 즉, 카테고리들간의 종속변수에 대한 평균값 차이가 큰거부터 왼쪽. 
#왼쪽변수일수록 종속변수에 대한 영향력이 크다. 카테고리들 간의 분포가 유의미 하게 차이가 나기 때문이다.


In [ ]:
def spearman(frame, features):
    spr = pd.DataFrame()
    spr['feature'] = features
    spr['spearman'] = [frame[f].corr(frame['Loss on Dispostion'], 'spearman') for f in features]
    spr = spr.sort_values('spearman')
    plt.figure(figsize=(6, 0.25*len(features)))
    sns.barplot(data=spr, y='feature', x='spearman', orient='h')
    
features = quantitative + qual_encoded_C
spearman(df_C, features)

In [ ]:
#변수 그룹별 상관관계
#Correlation matrix

plt.figure(figsize=(10, 8)) 
plt.figure(1)
corr = df[quantitative+['Loss on Dispostion']].corr()
sns.heatmap(corr)

plt.figure(figsize=(10, 8))
plt.figure(2)
corr = df[qual_encoded_C+['Loss on Dispostion']].corr()
sns.heatmap(corr)

plt.figure(figsize=(10, 8))
plt.figure(3)
corr = pd.DataFrame(np.zeros([len(quantitative)+1, len(qual_encoded_C)+1]), index=quantitative+['Loss on Dispostion'], columns=qual_encoded_C+['Loss on Dispostion'])
for q1 in quantitative+['Loss on Dispostion']:
    for q2 in qual_encoded_C+['Loss on Dispostion']:
        corr.loc[q1, q2] = df[q1].corr(df[q2])
sns.heatmap(corr)

In [ ]:
def boxplot(x, y, **kwargs):
    sns.boxplot(x=x, y=y)
    x=plt.xticks(rotation=90)
f = pd.melt(df, id_vars=['Loss on Dispostion'], value_vars=qualitative)
g = sns.FacetGrid(f, col="variable",  col_wrap=2, sharex=False, sharey=False, size=5)
g = g.map(boxplot, "value", "Loss on Dispostion")

In [ ]:
# pairplot 에서 loan_condition_E값으로 melt 하기 위해서는 겹쳐지니 뺸다.
quantitative.remove('Loss on Dispostion')

In [ ]:
def pairplot(x, y, **kwargs):
    ax = plt.gca()
    ts = pd.DataFrame({'time': x, 'val': y})
    ts = ts.groupby('time').mean()
    ts.plot(ax=ax)
    plt.xticks(rotation=90)
    
f = pd.melt(df_C, id_vars=['Loss on Dispostion'], value_vars=quantitative+qual_encoded_C)
g = sns.FacetGrid(f, col="variable",  col_wrap=2, sharex=False, sharey=False, size=5)
g = g.map(pairplot, "value", "Loss on Dispostion")

In [ ]:
df_C.describe()

In [ ]:
# Extract features and labels 

Label = df_C['Loss on Dispostion']
features = df_C.drop('Loss on Dispostion', axis = 1)

### VIF 검사

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(
    df_C.values, i) for i in range(df_C.shape[1])]
vif["features"] = df_C.columns
vif

In [ ]:
import statsmodels.api as sm
model1 = sm.OLS(Label, features)
result1 = model1.fit()
print(result1.summary())

In [ ]:
corr=df_C.corr()
corr.loc[:,:] = np.tril(corr, k=-1)   # below main lower triangle of an array
corr = corr.stack()
corr[(corr > 0.65) | (corr < -0.65)]

In [ ]:
df_C= df_C.drop('interest_payments_E',axis=1)

In [ ]:
# Extract features and labels 

Label = df_C['Loss on Dispostion']
features = df_C.drop('Loss on Dispostion', axis = 1)

In [ ]:

model1 = sm.OLS(Label, features)
result1 = model1.fit()
print(result1.summary())

In [ ]:
corr=df_C.corr()
corr.loc[:,:] = np.tril(corr, k=-1)   # below main lower triangle of an array
corr = corr.stack()
corr[(corr > 0.6) | (corr < -0.6)]

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(
    df_C.values, i) for i in range(df_C.shape[1])]
vif["features"] = df_C.columns
vif

In [ ]:
df_C= df_C.drop('InterestRate',axis=1)

In [ ]:
# Extract features and labels 

Label = df_C['Loss on Dispostion']
features = df_C.drop('Loss on Dispostion', axis = 1)

In [ ]:
# Training and Testing Sets
from sklearn.model_selection import train_test_split

# 데이터를 훈련+검증 세트 그리고 테스트 세트로 분할
X_trainval, X_test, y_trainval, y_test = train_test_split(
    features, Label, random_state=0)
# 훈련+검증 세트를 훈련 세트와 검증 세트로 분할
X_train, X_valid, y_train, y_valid = train_test_split(
    X_trainval, y_trainval, random_state=1)
print("train 세트의 크기: {}   validate 세트의 크기: {}   test 세트의 크기:"
      " {}\n".format(X_train.shape[0], X_valid.shape[0], X_test.shape[0]))

In [ ]:
### 데이터 전처리 - 데이터 표준화 작업 (scaling)
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler  # 0~1 사이의 숫자로 바꾼다. 양수가 필요할때 변환

sc = RobustScaler()
transformer = sc.fit(X_train)
transformer

sc2= StandardScaler()
transformer2 = sc2.fit(X_train)

sc3= MinMaxScaler()
transformer3 = sc3.fit(X_train)

# 4분위수를 이용한 표준화 스케일 방법인 RobustScaler를 이용.이상치 영향 약화. 

In [ ]:
#위에서 training set으로 fit 시킨 평균,분산 값을 이용하여 각 세트의 독립변수(X)들을 변환시킨다. 
X_train_std=transformer.transform(X_train)
X_valid_std=transformer.transform(X_valid)
X_test_std=transformer.transform(X_test)


X_train_std2=transformer2.transform(X_train)
X_valid_std2=transformer2.transform(X_valid)
X_test_std2=transformer2.transform(X_test)

X_train_std3=transformer3.transform(X_train)
X_valid_std3=transformer3.transform(X_valid)
X_test_std3=transformer3.transform(X_test)

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(
    X_train.values, i) for i in range(X_train.shape[1])]
vif["features"] = X_train.columns
vif

In [ ]:
# import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
import copy
import chart_studio.plotly as py

from plotly.offline import *

In [ ]:
## target Proportion ##
cnt = df_C['Loss on Dispostion'].value_counts()
tr_prop = go.Bar(
    x=cnt.index,
    y=cnt.values,
    marker=dict(
        color=cnt.values,
        colorscale = 'Picnic',
        reversescale = True
    ),
)

layout = go.Layout(
    title='Default Proportion',
    font=dict(size=18)
)

data = [tr_prop]
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename="TargetProp")

## target distribution ##
labels = (np.array(cnt.index))
sizes = (np.array((cnt / cnt.sum())*100))

tr_pie = go.Pie(labels=labels, values=sizes)
layout = go.Layout(
    title='Default Pie chart',
    font=dict(size=18),
    width=600,
    height=600,
)
data = [tr_pie]
fig = go.Figure(data=data, layout=layout)

iplot(fig, filename="pie_ty")


In [ ]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

missing_values_table(df_C)

In [ ]:
from scipy.stats import spearmanr

labels = []
values = []
for col in df_C.columns:
    if col not in ["Loss on Dispostion"]:
        labels.append(col)
        values.append(spearmanr(df_C[col].values, df_C["Loss on Dispostion"].values)[0])
corr_df = pd.DataFrame({'col_labels':labels, 'corr_values':values})
corr_df = corr_df.sort_values(by='corr_values')
 
ind = np.arange(corr_df.shape[0])
width = 0.9
fig, ax = plt.subplots(figsize=(12,30))
rects = ax.barh(ind, np.array(corr_df.corr_values.values), color='g')
ax.set_yticks(ind)
ax.set_yticklabels(corr_df.col_labels.values, rotation='horizontal')
ax.set_xlabel("Correlation coefficient")
ax.set_title("Correlation coefficient of the variables")
plt.show()

In [ ]:
cols_to_use = corr_df.ix[(corr_df['corr_values'] < -0.05) | (corr_df['corr_values']>0.05)].col_labels.tolist()

temp_df = df_C[cols_to_use]
corrmat = temp_df.corr(method='spearman')
f, ax = plt.subplots(figsize=(20, 20))

# Draw the heatmap using seaborn
sns.heatmap(corrmat, vmax=1., square=True, cmap="YlGnBu", annot=True)
plt.title("Important variables correlation map", fontsize=15)
plt.show()

In [ ]:

print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)

### Modeling

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc_model = RandomForestClassifier(random_state=0).fit(X_train, y_train)

#Feature importance by eli5
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(rfc_model, random_state=1).fit(X_valid, y_valid)
eli5.show_weights(perm, feature_names = X_valid.columns.tolist())

### PermutationImportance
Interpreting Permutation Importances The values towards the top are the most important features, and those towards the bottom matter least.

The first number in each row shows how much model performance decreased with a random shuffling (in this case, using "accuracy" as the performance metric).

Like most things in data science, there is some randomness to the exact performance change from a shuffling a column. We measure the amount of randomness in our permutation importance calculation by repeating the process with multiple shuffles. The number after the ± measures how performance varied from one-reshuffling to the next.

You'll occasionally see negative values for permutation importances. In those cases, the predictions on the shuffled (or noisy) data happened to be more accurate than the real data. This happens when the feature didn't matter (should have had an importance close to 0), but random chance caused the predictions on shuffled data to be more accurate. This is more common with small datasets, like the one in this example, because there is more room for luck/chance.

In [ ]:
from sklearn.feature_selection import SelectFromModel

# perm.feature_importances_ attribute is now available, it can be used
# for feature selection - let's e.g. select features which increase
# accuracy by at least 0.05:
sel = SelectFromModel(perm, threshold=0.05, prefit=True)
X_trans = sel.transform(X_train_std)
X_trans



In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(random_state=42)
forest.fit(X_train_std, y_train)

In [ ]:
print("훈련 세트 정확도: {:.3f}".format(forest.score(X_train_std, y_train)))
print("테스트 세트 정확도: {:.3f}".format(forest.score(X_valid_std, y_valid)))

In [ ]:
print("훈련 세트 정확도: {:.3f}".format(forest.score(X_train_std, y_train)))
print("테스트 세트 정확도: {:.3f}".format(forest.score(X_test_std, y_test)))

In [ ]:
from sklearn.metrics import confusion_matrix

forest.fit(X_train_std, y_train)
y_pred = forest.predict(X_test_std)
cm = confusion_matrix(y_test, y_pred)

print(cm)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred, target_names=['current', 'default']))

In [ ]:
from sklearn.metrics import roc_auc_score
y_hat = forest.predict(X_test_std)
score = roc_auc_score(y_test, y_hat)
print("Overall AUC: {:.3f}" .format(score))

In [ ]:
# Output feature importance coefficients, map them to their feature name, and sort values
coef = pd.Series(forest.feature_importances_, index = X_train.columns).sort_values(ascending=False)

plt.figure(figsize=(10, 5))
coef.head(25).plot(kind='bar')
plt.title('Feature Significance')
plt.tight_layout()

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbrt = GradientBoostingClassifier(random_state=42)
gbrt.fit(X_train_std, y_train)



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

rfc_model = GradientBoostingClassifier(random_state=42).fit(X_train_std, y_train)

#Feature importance by eli5
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(rfc_model, random_state=1).fit(X_valid_std, y_valid)
eli5.show_weights(perm, feature_names = X_valid.columns.tolist())


In [ ]:
print("훈련 세트 정확도: {:.3f}".format(gbrt.score(X_train_std, y_train)))
print("테스트 세트 정확도: {:.3f}".format(gbrt.score(X_valid_std, y_valid)))

In [ ]:
# 최종 테스트 (위에서 가장 성능좋은것 실행후 하자 이름이 모두 gbrt로 같기 때문.)
print("훈련 세트 정확도: {:.3f}".format(gbrt.score(X_train_std, y_train)))
print("테스트 세트 정확도: {:.3f}".format(gbrt.score(X_test_std, y_test)))

In [ ]:
# Get numerical feature importances

feature_list = list(X_train.columns)
importances = list(gbrt.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]

# list of x locations for plotting
x_values = list(range(len(importances)))
# Make a bar chart
plt.bar(x_values, importances, orientation = 'vertical', color = 'r', edgecolor = 'k', linewidth = 1.2)
# Tick labels for x axis
plt.xticks(x_values, feature_list, rotation='vertical')
# Axis labels and title
plt.ylabel('Importance'); plt.xlabel('Variable'); plt.title('Variable Importances');

In [ ]:
# List of features sorted from most to least important
sorted_importances = [importance[1] for importance in feature_importances]
sorted_features = [importance[0] for importance in feature_importances]
# Cumulative importances
cumulative_importances = np.cumsum(sorted_importances)
# Make a line graph
plt.plot(x_values, cumulative_importances, 'g-')
# Draw line at 80% of importance retained
plt.hlines(y = 0.80, xmin=0, xmax=len(sorted_importances), color = 'r', linestyles = 'dashed')
# Format x ticks and labels
plt.xticks(x_values, sorted_features, rotation = 'vertical')
# Axis labels and title
plt.xlabel('Variable'); plt.ylabel('Cumulative Importance'); plt.title('Cumulative Importances');

In [ ]:

from xgboost import XGBClassifier

# fit model no training data
xgb = XGBClassifier()
xgb.fit(X_train_std, y_train)


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

rfc_model = XGBClassifier(random_state=42).fit(X_train_std, y_train)

#Feature importance by eli5
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(rfc_model, random_state=1).fit(X_valid_std, y_valid)
eli5.show_weights(perm, feature_names = X_valid.columns.tolist())


In [ ]:
print("훈련 세트 정확도: {:.3f}".format(xgb.score(X_train_std, y_train)))
print("테스트 세트 정확도: {:.3f}".format(xgb.score(X_valid_std, y_valid)))

In [ ]:
print("훈련 세트 정확도: {:.3f}".format(xgb.score(X_train_std, y_train)))
print("테스트 세트 정확도: {:.3f}".format(xgb.score(X_test_std, y_test)))

In [ ]:
from sklearn.metrics import accuracy_score

# make predictions for test data
y_pred = xgb.predict(X_test_std)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
predictions.count(0)

In [ ]:
predictions.count(1)

In [ ]:
# Get numerical feature importances

feature_list = list(X_train.columns)
importances = list(xgb.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]

# list of x locations for plotting
x_values = list(range(len(importances)))
# Make a bar chart
plt.bar(x_values, importances, orientation = 'vertical', color = 'r', edgecolor = 'k', linewidth = 1.2)
# Tick labels for x axis
plt.xticks(x_values, feature_list, rotation='vertical')
# Axis labels and title
plt.ylabel('Importance'); plt.xlabel('Variable'); plt.title('Variable Importances');

In [ ]:
# List of features sorted from most to least important
sorted_importances = [importance[1] for importance in feature_importances]
sorted_features = [importance[0] for importance in feature_importances]
# Cumulative importances
cumulative_importances = np.cumsum(sorted_importances)
# Make a line graph
plt.plot(x_values, cumulative_importances, 'g-')
# Draw line at 80% of importance retained
plt.hlines(y = 0.80, xmin=0, xmax=len(sorted_importances), color = 'r', linestyles = 'dashed')
# Format x ticks and labels
plt.xticks(x_values, sorted_features, rotation = 'vertical')
# Axis labels and title
plt.xlabel('Variable'); plt.ylabel('Cumulative Importance'); plt.title('Cumulative Importances');

In [ ]:
from sklearn.metrics import confusion_matrix

gbrt.fit(X_train_std, y_train)
y_pred = gbrt.predict(X_test_std)
cm = confusion_matrix(y_test, y_pred)

print(cm)

In [ ]:
(cm[0,0]+cm[1,1])/np.sum(cm)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred, target_names=['current', 'default']))

In [ ]:
from sklearn.metrics import roc_auc_score
y_hat = gbrt.predict(X_test_std)
score = roc_auc_score(y_test, y_hat)
print("Overall AUC: {:.3f}" .format(score))

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [ ]:
print('Mean squared error: ', mse)
print('R2 score: ', r2)

In [ ]:
from sklearn.metrics import roc_auc_score
y_hat = gbrt.predict(X_test_std)
score = roc_auc_score(y_test, y_hat)
print("Overall AUC: {:.3f}" .format(score))

In [ ]:
from sklearn import metrics
from sklearn.metrics import roc_curve

y_pred_proba = gbrt.predict_proba(X_valid_std)[::,1]
fpr, tpr, _ = roc_curve(y_valid, y_pred_proba, pos_label=1) 
# fpr, tpr, _ = metrics.roc_curve(y_valid,  y_pred_proba)
auc = metrics.roc_auc_score(y_valid, y_pred_proba)
plt.plot(fpr,tpr,label="ROC Curve, auc="+str(auc))
plt.legend(loc=4)
plt.show()


In [ ]:
from sklearn.metrics import roc_curve
    
y_pred_prob = gbrt.predict_proba(X_valid_std)[:,1]

fpr, tpr, thresholds = roc_curve(y_valid, y_pred_prob, pos_label=1)
auc = metrics.roc_auc_score(y_valid, y_pred_prob)

# create plot
plt.plot(fpr, tpr, label='ROC curve,  auc='+str(auc))
plt.plot([0, 1], [0, 1], 'k--', label='Random guess')
_ = plt.xlabel('False Positive Rate')
_ = plt.ylabel('True Positive Rate')
_ = plt.title('ROC Curve')
_ = plt.xlim([-0.02, 1])
_ = plt.ylim([0, 1.02])
_ = plt.legend(loc="lower right")


# save figure
#plt.savefig('roc_curve.png', dpi=200)

In [ ]:
from sklearn.metrics import roc_curve
    
y_pred_prob = gbrt.predict_proba(X_valid_std)[:,0]

fpr, tpr, thresholds = roc_curve(y_valid, y_pred_prob, pos_label=0)
auc = metrics.roc_auc_score(y_valid, y_pred_prob)

# create plot
plt.plot(fpr, tpr, label='ROC curve,  auc='+str(auc))
plt.plot([0, 1], [0, 1], 'k--', label='Random guess')
_ = plt.xlabel('False Positive Rate')
_ = plt.ylabel('True Positive Rate')
_ = plt.title('ROC Curve')
_ = plt.xlim([-0.02, 1])
_ = plt.ylim([0, 1.02])
_ = plt.legend(loc="lower right")


# save figure
#plt.savefig('roc_curve.png', dpi=200)

In [ ]:
from sklearn.metrics import precision_recall_curve 
from sklearn.metrics import average_precision_score

precision_score= average_precision_score(y_valid, y_pred_prob,pos_label=0)
precision, recall, thresholds = precision_recall_curve(y_valid, y_pred_prob, pos_label=0)

# create plot
plt.plot(precision, recall, label='Precision-recall curve,  AP='+str(precision_score))
_ = plt.xlabel('Recall')
_ = plt.ylabel('Precision')
_ = plt.title('Precision-recall curve')
_ = plt.xlim([0.735, 1])
_ = plt.ylim([0, 1.02])
_ = plt.legend(loc="lower left")

# save figure
# plt.savefig('precision_recall.png', dpi=200)

In [ ]:
from sklearn.metrics import precision_recall_curve 
from sklearn.metrics import average_precision_score

precision_score= average_precision_score(y_valid, y_pred_prob,pos_label=1)
precision, recall, thresholds = precision_recall_curve(y_valid, y_pred_prob, pos_label=1)

# create plot
plt.plot(precision, recall, label='Precision-recall curve,  AP='+str(precision_score))
_ = plt.xlabel('Recall')
_ = plt.ylabel('Precision')
_ = plt.title('Precision-recall curve')
_ = plt.xlim([0.735, 1])
_ = plt.ylim([0, 1.02])
_ = plt.legend(loc="lower left")

# save figure
# plt.savefig('precision_recall.png', dpi=200)

In [ ]:
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(y_valid, y_pred_prob,pos_label=0)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))


from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
#from sklearn.utils.fixes import signature
from inspect import signature

precision, recall, _ = precision_recall_curve(y_valid, y_pred_prob, pos_label=0)

# In matplotlib < 1.5, plt.fill_between does not have a 'step' argument
step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision))

In [ ]:
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(y_valid, y_pred_prob,pos_label=1)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))


from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
#from sklearn.utils.fixes import signature
from inspect import signature

precision, recall, _ = precision_recall_curve(y_valid, y_pred_prob, pos_label=1)

# In matplotlib < 1.5, plt.fill_between does not have a 'step' argument
step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision))

### Ensemble Voting

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier


log_clf = LogisticRegression(random_state=42)
rnd_clf = RandomForestClassifier(random_state=42)
svm_clf = SVC(random_state=42)
knn_clf = KNeighborsClassifier(n_neighbors=15)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf),('knn', knn_clf) ],
    voting='hard')
voting_clf.fit(X_train_std, y_train)

In [ ]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, knn_clf, voting_clf):
    clf.fit(X_train_std, y_train)
    y_pred = clf.predict(X_valid_std)
    print(clf.__class__.__name__, accuracy_score(y_valid, y_pred))

In [ ]:
log_clf = LogisticRegression(random_state=42)
rnd_clf = RandomForestClassifier(random_state=42)
svm_clf = SVC(random_state=42)
knn_clf = KNeighborsClassifier(n_neighbors=15)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf),('knn', knn_clf) ],
    voting='soft')
voting_clf.fit(X_train_std, y_train)

In [ ]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, knn_clf, voting_clf):
    clf.fit(X_train_std, y_train)
    y_pred = clf.predict(X_valid_std)
    print(clf.__class__.__name__, accuracy_score(y_valid, y_pred))

#### voting='soft' 인 경우 각 분류기의 예측값(레이블)의 확률을 가지고 평균을 구한 뒤, 평균이 가장 높은 클래스로 최종 앙상블 예측이 이루어진다. 이러한 방법을 간접 투표(soft voting)이라 한다. 이 경우에는 레이블의 확률값을 구할수 없기에 에러가 난다.

### cross-validation

In [ ]:
### 데이터 전처리 - 데이터 표준화 작업 (scaling)
from sklearn.preprocessing import RobustScaler

sc = RobustScaler()
transformer = sc.fit(features)
transformer

#위에서 training set으로 fit 시킨 평균,분산 값을 이용하여 각 세트의 독립변수(X)들을 변환시킨다. 
features_std=transformer.transform(features)


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

skf=StratifiedKFold(n_splits=10, shuffle=True,random_state=0)
score=cross_val_score(gbrt,features_std, Label, cv=skf)
print(score)
print(score.mean())

In [ ]:
# cross-validation 을 사용하여 평가한 모델을 이용하여 예측

from sklearn.model_selection import cross_val_predict
y_predict = cross_val_predict(gbrt,features_std, Label, cv=skf)
print(y_predict)

In [ ]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(gbrt,features_std, Label, cv=skf)
print(sorted(cv_results.keys()))
#print(cv_results['train_score'])
print(cv_results['test_score'])
print(cv_results['test_score'].mean())


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(Label, y_predict, target_names=['current', 'default']))

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(Label, y_predict)

print(cm)

In [ ]:
(cm[0,0]+cm[1,1])/np.sum(cm)

In [ ]:
from sklearn.metrics import roc_auc_score
y_hat = gbrt.predict(X_test_std)
score = roc_auc_score(y_test, y_hat)
print("Overall AUC: {:.3f}" .format(score))

In [ ]:
# df_v['crossval_result']=y_predict.tolist()
# df_v['compare'] = np.where(df['Loss on Dispostion'] == df_v['crossval_result'], '0', '1')
# df_v['compare'].value_counts()


In [ ]:
# 4878/len(df_cv['compare'])

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold

skf=StratifiedKFold(n_splits=10, shuffle=True,random_state=0)
cv_results = cross_validate(gbrt,features_std, Label, cv=skf, return_estimator =True)


for idx,estimator in enumerate(cv_results['estimator']):
    print("Features sorted by their score for estimator {}:".format(idx))
    feature_importances = pd.DataFrame(estimator.feature_importances_,
                                       index = features.columns,
                                        columns=['importance']).sort_values('importance', ascending=False)
    print(feature_importances)


### 결론: cross-validation을 사용하여 predict 해보니기존의 train,test set 나눈후 모델 적용시킨 결과보다  default 예측이 더 높았다. 그것은 전체 데이터수가 적은데다가 또한 임의로 한번 데이터셋을 나누었을때 default 데이터가 균등히 섞이지 않았을 가능성이 높아서 default 예측 성능이 떨어진 것이다.  이런경우 cross-validation, 특히 StratifiedKFold 적용의 효과를 볼수있다.

In [ ]:
plt.figure(figsize=(12,6))

plt.subplot(121)
g = sns.distplot(df["Loss on Dispostion"])
g.set_xlabel("", fontsize=12)
g.set_ylabel("Frequency Dist", fontsize=12)
g.set_title("Frequency Distribuition", fontsize=20)

plt.subplot(122)
g1 = sns.violinplot(y="Loss on Dispostion", data=df, 
               inner="quartile", palette="hls")
g1.set_xlabel("", fontsize=12)
g1.set_ylabel("Amount Dist", fontsize=12)
g1.set_title("Amount Distribuition", fontsize=20)

plt.show()

In [ ]:
df['Loss on Dispostion'].unique()

In [ ]:
df_p=df_C[['DealerCode_E','ProductSeries_E','Vehicle_Model_E','CurrentKms','Payment','Customers_Applicant_Age','LTV','Customers_Applicant_AnnualTotal','TotalCalculatedDebt','Loss on Dispostion']]
df_p.head()

In [ ]:
#df_p.sort_values(by='loan_condition_E')
df_p.groupby(df_p['Loss on Dispostion']).mean()

In [ ]:
df_C.DealerCode_E.max()

In [ ]:

df_D=pd.concat([df_C.DealerCode_E,df.DealerCode], axis=1)
df_D.sort_values(by='DealerCode_E')

In [ ]:
df_count=df_D.groupby(df_D.DealerCode_E).count().sort_values('DealerCode', ascending=False)
df_count

In [ ]:
df_s=pd.concat([df_C.ProductSeries_E,df.ProductSeries], axis=1)
df_s.sort_values(by='ProductSeries_E')

In [ ]:
df_v=pd.concat([df_C.Vehicle_Model_E,df.Vehicle_Model], axis=1)
df_v.sort_values(by='Vehicle_Model_E')

### SMOTE Over-Sampling
As we have more records for target '0', I am going to over sample the target '1' to the same level as target '0' which is basically oversampling the least class.


In [ ]:
#Using SMOTE for class imbalance in target
from imblearn.over_sampling import SMOTE
from collections import Counter

sm = SMOTE(random_state=42)
X_resamp, y_resamp = sm.fit_sample(X_train_std, y_train)
print('Resampled dataset shape %s' % Counter(y_resamp))
X_resamp = pd.DataFrame(X_resamp)
y_resamp = pd.DataFrame({"target": y_resamp})

In [ ]:
print(classification_report(y_resamp, forest.predict(X_resamp),target_names=['current', 'default']))

In [ ]:
print(classification_report(y_resamp, gbrt.predict(X_resamp),target_names=['current', 'default']))

### Light GBM

In [ ]:
params = {'num_leaves': 9,
         'min_data_in_leaf': 42,
         'objective': 'binary',
         'max_depth': 16,
         'learning_rate': 0.0123,
         'boosting': 'gbdt',
         'bagging_freq': 5,
         'bagging_fraction': 0.8,
         'feature_fraction': 0.8201,
         'bagging_seed': 11,
         'reg_alpha': 1.728910519108444,
         'reg_lambda': 4.9847051755586085,
         'random_state': 42,
         'metric': 'auc',
         'verbosity': -1,
         'subsample': 0.81,
         'min_gain_to_split': 0.01077313523861969,
         'min_child_weight': 19.428902804238373,
         'num_threads': 4}

In [ ]:
# Create parameters to search
gridParams = {
    'learning_rate': [0.005],
    'n_estimators': [40],
    'num_leaves': [6,8,12,16],
    'boosting_type' : ['gbdt'],
    'objective' : ['binary'],
    'random_state' : [501], # Updated from 'seed'
    'colsample_bytree' : [0.65, 0.66],
    'subsample' : [0.7,0.75],
    'reg_alpha' : [1,1.2],
    'reg_lambda' : [1,1.2,1.4],
    }


In [ ]:
%%time
import time
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
fold_n=3
folds = StratifiedKFold(n_splits=fold_n, shuffle=True, random_state=30)
y_pred_lgb = np.zeros(len(X_test_std))
for fold_n, (train_index, valid_index) in enumerate(folds.split(X_resamp,y_resamp)):
    print('Fold', fold_n, 'started at', time.ctime())
    X_train, X_valid = X_resamp.iloc[train_index], X_resamp.iloc[valid_index]
    y_train, y_valid = y_resamp.iloc[train_index], y_resamp.iloc[valid_index]
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid)
        
    lgb_model = lgb.train(params,train_data,num_boost_round=20000,
                    valid_sets = [train_data, valid_data],verbose_eval=300,early_stopping_rounds = 200)
            
    y_pred_lgb += lgb_model.predict(X_test_std, num_iteration=lgb_model.best_iteration)/3

In [ ]:
y_pred_lgb

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred_lgb)
r2 = r2_score(y_test, y_pred_lgb)

In [ ]:
print('Mean squared error: ', mse)
print('R2 score: ', r2)

In [ ]:
from sklearn.metrics import roc_auc_score
y_hat = lgb_model.predict(X_test_std)
score = roc_auc_score(y_test, y_hat)
print("Overall AUC: {:.3f}" .format(score))

In [ ]:
clf = lgb.LGBMClassifier(class_weight='balanced',drop_rate=0.9, min_data_in_leaf=100, max_bin=255,
                                 n_estimators=500,min_sum_hessian_in_leaf=1,importance_type='gain',learning_rate=0.1,bagging_fraction = 0.85,
                                 colsample_bytree = 1.0,feature_fraction = 0.1,lambda_l1 = 5.0,lambda_l2 = 3.0,max_depth =  9,
                                 min_child_samples = 55,min_child_weight = 5.0,min_split_gain = 0.1,num_leaves = 45,subsample = 0.75)  

In [ ]:
clf

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True)
# partially based on https://www.kaggle.com/c0conuts/xgb-k-folds-fastai-pca
predicts = []
for train_index, test_index in kf.split(X_resamp,y_resamp):
    print("###")
    X_train, X_valid = X_resamp.iloc[train_index], X_resamp.iloc[valid_index]
    y_train, y_valid = y_resamp.iloc[train_index], y_resamp.iloc[valid_index]
    
    clf.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], 
            early_stopping_rounds = 20)
    predicts.append(clf.predict(X_test_std))

In [ ]:
predicts

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# sorted(zip(clf.feature_importances_, X.columns), reverse=True)
feature_imp = pd.DataFrame(sorted(zip(clf.feature_importances_,features.columns)), columns=['Value','Feature'])

plt.figure(figsize=(30, 30))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features (avg over folds)',fontsize=25)
plt.tight_layout()

#plt.rcParams.update({'font.size': 10})
plt.show()
plt.savefig('lgbm_importances-01.png')

In [ ]:
final_result = pd.concat([y_test.reset_index(drop=True), pd.DataFrame(y_pred_lgb)], axis = 1)

In [ ]:
final_result.columns = ['label','predict']

In [ ]:
final_result

In [ ]:
sns.regplot(x = 'label', y = 'predict', data = final_result)

In [ ]:
final_result.groupby(['label']).describe()

#### parameter engineering 모델 성능 튜닝

In [ ]:
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
print(clf.get_params())

In [ ]:
#RandomizedSearchCV 를 위한 파라미터 조건 설정

from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 550, num = 20)]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 5)]
max_depth.append(None)
# Minimum number of child samples 
min_child_samples = [50, 55, 60]
# drop rate
drop_rate = [0.7, 0.8, 0.9]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               
               'max_depth': max_depth,
               'min_child_samples': min_child_samples,
               
               'drop_rate': drop_rate}

print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters

# Random search of parameters, using 3 fold cross validation, 
# search across (n_iter*cv) different combinations, and use all available cores
clf_random = RandomizedSearchCV(estimator=clf, param_distributions=random_grid,
                              n_iter = 10,  
                              cv = 3, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)

# Fit the random search model
clf_random.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], 
            early_stopping_rounds = 20);

# n_iter, cv가 주요 하이퍼파라미터이다.

In [ ]:
clf_random.best_params_

In [ ]:
best_random = clf_random.best_estimator_

y_pred=best_random.predict(X_test_std)
print('Accuracy =', accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.model_selection import GridSearchCV

# Create the parameter grid based on the results of random search 

param_grid = {
    'bootstrap': [True, False],
    'max_depth': [30, 50],
    'drop_rate': [0.8, 0.9],
    'min_child_samples': [55,65],
    'n_estimators': [130,155]
}

# Create a base model
# clf

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = clf, param_grid = param_grid, 
                          cv = 2, n_jobs = -1, verbose = 2, return_train_score=True)

# Fit the random search model
grid_search.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], 
            early_stopping_rounds = 20);


In [ ]:
grid_search.best_params_

In [ ]:
best_grid = grid_search.best_estimator_

y_pred=best_grid.predict(X_test_std)
print('Accuracy =', accuracy_score(y_test, y_pred))

## clustering

In [ ]:
inputs=df_p.iloc[:,0:9]
inputs.head()

In [ ]:
### 데이터 전처리 - 데이터 표준화 작업 (scaling)
from sklearn.preprocessing import RobustScaler

sc = RobustScaler()
transformer = sc.fit(inputs)
transformer

inputs_std=X_train_std=transformer.transform(inputs)


In [ ]:

Kmeans=KMeans(n_clusters=2)
Kmeans.fit(inputs_std)
Kmeans.labels_

In [ ]:
colormap=np.array(['red','blue','green','orange'])
plt.scatter(inputs.CurrentKms, inputs.Payment, c=colormap[Kmeans.labels_],s=20)
plt.title('kmeans cluster')
plt.show()

In [ ]:
print(Kmeans)
print(Kmeans.cluster_centers_)

In [ ]:
#SSE(오차제곱합) 최소되도록 최적 k값 찾기 
def elbow(x):
    sse=[]
    for i in range(1,11):
        y=KMeans(n_clusters=i, init='k-means++', random_state=0)
        y.fit(x)
        sse.append(y.inertia_)

    plt.plot(range(1,11),sse,marker='o')
    plt.xlabel('cluster number')
    plt.ylabel('SSE')
    plt.show()

elbow(inputs)

In [ ]:
#정확한 답(클러스터의 갯수 및 소속)을 검사위해 clustring의 품질을 정량적으로 계산
#Silhouette Coefficient
#실루엣 계수가 1에 가까울수록 클러스터 n 최적화

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm


X=inputs
range_n_clusters = [2, 3, 4, 5, 6, 7]

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 100])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    
    for i in range(n_clusters):
        
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        #color = cm.spectral(float(i) / n_clusters)
        cmap = cm.get_cmap("Spectral")
        colors = cmap(float(i) / n_clusters)
        
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=colors, edgecolor=colors, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i+1))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    #colors = cm.spectral(cluster_labels.astype(float) / n_clusters)
    
    #color = cm.spectral(float(i) / n_clusters)
    cmap = cm.get_cmap("Spectral")
    colors = cmap(float(i) / n_clusters)
        
    ax2.scatter(X.iloc[:, 3], X.iloc[:, 4], marker='.', s=30, lw=0, alpha=0.7,
                c=colors)

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(centers[:, 3], centers[:, 4],
                marker='o', c="yellow", alpha=1, s=200)

    for i, c in enumerate(centers):
        r=i+1
        ax2.scatter(c[3], c[4], marker='$%d$' % r, alpha=1, s=50)

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')

    plt.show()   

In [ ]:
Kmeans.labels_

In [ ]:
x=inputs_std #data type (df->array)
y=Kmeans.fit_predict(inputs_std)

In [ ]:
# y(집단)의 inputs 변수 두개의 분포를 보자.

plt.scatter(x[y==0,3],x[y==0,4],c='lightgreen', marker='s',label='clu1')
plt.scatter(x[y==1,3],x[y==1,4],c='orange', marker='o',label='clu2')
# plt.scatter(x[y==2,2],x[y==2,4],c='lightblue', marker='v',label='clu3')
# plt.scatter(x[y==3,2],x[y==3,4],c='b', marker='^',label='clu4')

# cluster.center
plt.scatter(Kmeans.cluster_centers_[:,3],Kmeans.cluster_centers_[:,4],c='red', marker='*',label='center')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
df_p['kmeansResult']=Kmeans.labels_



In [ ]:
df_p['compare'] = np.where(df_p['Loss on Dispostion'] == df_p['kmeansResult'], '0', '1')

In [ ]:
df_p['compare'].value_counts()

In [ ]:
4182/len(df_p['compare'])

In [ ]:
cm = confusion_matrix(df_p['Loss on Dispostion'], df_p['kmeansResult'])

print(cm)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(df_p['Loss on Dispostion'], df_p['kmeansResult'], target_names=['current', 'default']))

In [ ]:
import seaborn as sns 

sns.set(color_codes=True)

g = sns.clustermap(x,cmap="mako", robust=True)
plt.show()

In [ ]:
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
import numpy as np

In [ ]:
# some setting for this notebook to actually show the graphs inline
# you probably won't need this
%matplotlib inline
np.set_printoptions(precision=5, suppress=True)  # suppress scientific float notation

In [ ]:
# generate the linkage matrix
# agglomerative hierarchical clustering (bottom up) - Ward Linkage Method

Z = linkage(x, 'ward')

In [ ]:
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist

c, coph_dists = cophenet(Z, pdist(x))
c

In [ ]:
# calculate full dendrogram
plt.figure(figsize=(25, 10))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('customer index')
plt.ylabel('distance')
dendrogram(
    Z,
    leaf_rotation=90.,  # rotates the x axis labels
    leaf_font_size=8.,  # font size for the x axis labels
)
plt.show()